<img src="https://www.inf.utfsm.cl/images/slides/Departamento-de-Informtica_HORIZONTAL.png" title="Title text" width="80%" />

<hr style="height:2px;border:none"/>
<h1 align='center'> INF-398/578 Introducción al aprendizaje automático</h1>

<h4 align='center'><b>Tarea 3: Redes neuronales y ensamblados.</b></h4>


<h6 align='center'><b>Profesor:</b> Carlos Valle</h6>
<h6 align='center'><b>Ayudante:</b> Jean Aravena</h6>



<hr style="height:2px;border:none"/>

# **Tarea 3 📃**

## **Temas**  

* Redes neuronales.
* Ensamblados.
* Random Forest, XGBoost, CatBoost.


## **Formalidades** 

* Equipos de trabajo de 2 personas.
* El entregable debe ser este mismo Jupyter Notebook incluyendo todos los resultados, los gráficos realizados y las respuestas a las preguntas. 
* Se debe preparar una presentación del trabajo realizado y sus hallazgos. El presentador será elegido aleatoriamente y deberá apoyarse en el Jupyter Notebook que entregarán.
* Formato de entrega: Subir a aula el Jupyter Notebook con el nombre NombreGrupo_Tarea_3_ML_2022_1
* Fecha de entrega y presentaciones: 22 de Julio. Hora límite de entrega: 14:30.
 

<hr style="height:2px;border:none"/>

La tarea se divide en 3 partes:

1.   Redes Neuronales.
2.   Ensamblados.
3.   Desafío.


La tarea tiene ejemplos de códigos con los cuales pueden guiarse en gran parte, sin embargo, solo son guías y pueden ser creativos al momento de resolver la tarea. También en algunas ocasiones se hacen elecciones arbitrarias, ustedes pueden realizar otras elecciones con tal de que haya una pequeña justificación de por qué su elección es mejor o equivalente.

Recuerden intercalar su código con comentarios y utilizar celdas Markdown en caso de que sea necesario para realizar análisis, escribir fórmulas o realizar explicaciones que les parezca relevante para justificar sus procedimientos. 

Noten que en general cuando se les pide elegir algo o proponer algo no se evaluará mucho la elección en sí, en cambio la argumentación detrás de la elección será lo más ponderado.

*Se recomienda el uso de Google Colab para realizar la tarea.*

# Librerias 📚
 

Agregar cualquier otra librería que requiera para el desarrollo de la tarea.

In [ ]:
!pip install -q -U catboost
!pip install -q -U keras-tuner

In [ ]:
import os
import re
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from tensorflow.keras.layers import Input, Dense, Layer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError, Loss
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import median_absolute_error, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.decomposition import PCA

sns.set_theme()
warnings.filterwarnings('ignore')

# Utils ⚙️  

In [ ]:
print("Mounting your Google Drive ...")

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

# Path to your folder
path = '/content/drive/MyDrive/Machine Learning/Data'

Mounting your Google Drive ...
Mounted at /content/drive


In [ ]:
# Set all seed.
def set_all_seed(seed=0):
  """Set the same seed for all the libraries that we use."""
  np.random.seed(seed)
  tf.random.set_seed(seed)

# Elegir cualquier semilla y siempre que se pueda utilizarla
# random_state = SEED
SEED = 2022

set_all_seed(seed=SEED)

# 1.1) Redes Neuronales 💣

<img src="https://drive.google.com/uc?id=12fnVpIB_qDJx_OVty7aWT1Ftw2o9r1Hl" width="60%"/>

[Referencia](https://purnasaigudikandula.medium.com/a-beginner-intro-to-neural-networks-543267bda3c8)

Para la creación de redes neuronales generalmente existen dos bandos:


*   `Pytorch`
*   `Tensorflow`

Por un lado, tenemos a `Pytorch` que es mayormente utilizado en investigación y por otro lado tenemos a `Tensorflow` que es mayormente utilizado en la industria. Ambos tienen la misma capacidad de abstracción y ambos son ampliamente aceptados y poseen una gran comunidad que los respalda.

Si nos vamos a una página bastante conocida como lo es [Papers With Code](https://paperswithcode.com/) podemos ver que desde la fecha `2018-03-01` hasta `2022-03-31` existe una tendencia creciente para el uso de `Pytorch` en términos de utilización en nuevos papers, es por ello por lo que en general si se va a realizar investigación se recomienda `Pytorch`, pero ambos son útiles y si sabes uno se hace más fácil cambiarse de uno a otro. En introducción a las redes neuronales se utiliza `Tensorflow`, es por ello que también emplearemos ese framework en esta tarea, pero como se mencionó anteriormente no importa mucho cual aprendas porque te puedes cambiar cuando sea necesario.


<img src="https://drive.google.com/uc?id=1xVqc0kPI1TXmhMmsMJnxOiUsGAaqiYGP" width="80%"/>


[Referencia](https://paperswithcode.com/trends)

## Carga y exploración de la data

In [ ]:
# Cargar la data que corresponde a un csv
PATH_TO_CSV = os.path.join(path, "tarea_3_house_price.csv")
df_data = pd.read_csv(PATH_TO_CSV)

<img src="https://drive.google.com/uc?id=13v7iUc8zmqlQ46aMfbtQ5dIEtDcdiNYx" width="50%"/>


Para la primera parte de la tarea trabajaremos con un dataset muy clásico en ML el cual es [House Sales in King County, USA](https://www.kaggle.com/datasets/harlfoxem/housesalesprediction), que viene de `Kaggle`.

La data corresponde a 21.613 instancias las cuales tienen relación con precios de casas en dólares.

El dataset cuenta con 20 features las cuales están diseñadas para identificar ciertos patrones y con ello tratar de predecir el precio de una casa.

Features:

* `id`: Unique ID for each home sold
* `date`: Date of the home sale
* `bedrooms`: Number of bedrooms
* `bathrooms`: Number of bathrooms, where .5 accounts for a room with a toilet  but no shower
* `sqft_living`: Square footage of the apartments interior living space
* `sqft_lot`: Square footage of the land space
* `floors`: Number of floors
* `waterfront`: A dummy variable for whether the apartment was overlooking the waterfront or not
* `view`: An index from 0 to 4 of how good the view of the property was
* `condition`: An index from 1 to 5 on the condition of the apartment,
* `grade`: An index from 1 to 13, where 1-3 falls short of building construction and design, 7 has an average level of construction and design, and 11-13 have a high quality level of construction and design.
* `sqft_above`: The square footage of the interior housing space that is above ground level
* `sqft_basement`: The square footage of the interior housing space that is below ground level
* `yr_built`: The year the house was initially built
* `yr_renovated`: The year of the house’s last renovation
* `zipcode`: What zipcode area the house is in
* `lat`: Lattitude
* `long`: Longitude
* `sqft_living15`: The square footage of interior housing living space for the nearest 15 neighbors
* `sqft_lot15`: The square footage of the land lots of the nearest 15 neighbors

Target:
* `price`: Price of each home sold

**Como es usual realice un pequeño EDA (Exploratory Data Analysis). Comente los resultados obtenidos** [2 Pts]

In [ ]:
### START CODE HERE ###


## Preprocesamiento

**Divida la data en training/validation/testing considerando un split 80/10/10 y realice un preprocesamiento a la data** [2 Pts] 

In [ ]:
### START CODE HERE ###


## Formas de crear una red neuronal

Existen tres formas distintas para crear una red neuronal utilizando `Tensorflow`:


1.   `Sequential API`
2.   `Functional API`
3.   `Model Subclassing`


Ordenados de menor a mayor en términos de flexibilidad, pero con el costo de trabajar a más bajo nivel.




### ¿Cómo preparar la data para utilizarla en una red neuronal?


Antes de crear la red aprenderemos a cómo preparar la data para ser utilizada posteriormente. Si bien existen múltiples formas, veremos el estándar que tiene Google en sus [tutoriales](https://www.tensorflow.org/tutorials) de `Tensorflow`. Lo cual consiste en hacer uso de [tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/DatasetSpec)

**Siga los pasos indicados a continuación considerando la data** [1 Pts]

Separe cada dataframe en features y target.


In [ ]:
### START CODE HERE ###


Crear los dataset de entrenamiento, validacion y testing utilizando `tf.data.Dataset.from_tensor_slices` considerando como parámetro la tupla X, y. Ejemplo: `tf.data.Dataset.from_tensor_slices(X_train.values, y_train.values)`


In [ ]:
### START CODE HERE ###


Podemos mostrar la estructura del dataset con un print o bien llamando a la variable.

In [ ]:
train_dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(24,), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>

También podemos mostrar un elemento en específico del dataset.

In [ ]:
for (X, y) in train_dataset:
  print(X)
  print(y)
  break

tf.Tensor(
[-4.54545455e-01 -1.87500000e-01 -6.10544218e-01 -9.88726944e-01
 -6.00000000e-01  1.00000000e+00  3.00000000e+00  7.00000000e+00
 -6.56942824e-01 -5.28735632e-01  7.80000000e+01  0.00000000e+00
  1.40000000e+01  2.07978124e-01 -2.27574751e-01 -4.15652174e-01
  1.02700000e+04  1.00000000e+00 -7.43144825e-01 -6.69130606e-01
  1.00000000e+00  6.12323400e-17  1.00000000e+00  0.00000000e+00], shape=(24,), dtype=float64)
tf.Tensor(688888.0, shape=(), dtype=float64)


`Tensorflow` funciona en base a tensores. Un `tensor` no es más ni menos que una matriz con cero o más dimensiones. Un `tensor` de dimension cero se llama escalar


En este caso vamos a mostrar tensores de 0, 1 y 2 dimensiones para comprender como varia el valor de shape en cada caso.

In [ ]:
tensor_zero_dimension = tf.Variable(1, tf.int32)
tensor_zero_dimension 

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=1>

Si el `tensor` es un escalar entonces vamos a tener `shape=()`.

In [ ]:
tensor_one_dimension = tf.Variable([2, 3], tf.int32)
tensor_one_dimension

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([2, 3], dtype=int32)>

Si el `tensor` es de una dimension vamos a tener `shape=(2, )` lo que indica que tenemos un `tensor` con una dimension y dos elementos.

In [ ]:
tensor_two_dimension = tf.Variable([[1, 2], [2, 3]], tf.int32)
tensor_two_dimension

<tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[1, 2],
       [2, 3]], dtype=int32)>

Si el `tensor` es de dos dimensiones vamos a tener `shape=(2, 2)` lo que indica que tenemos un `tensor` con dos dimension y dos elementos.


En conclusión si tenemos `shape=(x, y)` el valor `x` corresponde a la cantidad de elementos que existe en el tensor e `y` corresponde a la dimensionalidad de este. Cuando `shape=()` significa que tenemos un escalar lo que es igual a tener un tensor de dimensionalidad cero.

En caso de que queramos obtener el valor de un `Tensor` como usualmente se aprecia en `numpy`, tenemos que utilizar `X.numpy()` o bien `y.numpy()`.

In [ ]:
for (X, y) in train_dataset:
  print(X.numpy())
  print(y.numpy())
  break

[-4.54545455e-01 -1.87500000e-01 -6.10544218e-01 -9.88726944e-01
 -6.00000000e-01  1.00000000e+00  3.00000000e+00  7.00000000e+00
 -6.56942824e-01 -5.28735632e-01  7.80000000e+01  0.00000000e+00
  1.40000000e+01  2.07978124e-01 -2.27574751e-01 -4.15652174e-01
  1.02700000e+04  1.00000000e+00 -7.43144825e-01 -6.69130606e-01
  1.00000000e+00  6.12323400e-17  1.00000000e+00  0.00000000e+00]
688888.0


Continuando con el tutorial de la preparación de la data, el primer paso es tener el dataset en la forma de tupla `(X, y)`. Como ya tenemos el dataset de la forma correcta, el siguiente paso es preparar el dataset para ser usado por la red neuronal. Para ello utilicen la siguiente funcion:

In [ ]:
def preprocess_dataset(dataset, batch_size, size):
  """Preprocess dataset.

    :param dataset: Dataset para prepararlo para el entrenamiento
    :type dataset: tensorflow.python.data.ops.dataset_ops.TensorSliceDataset
    :param batch_size: Tamano del batch a utilizar para entrenamiento
    :type batch_size: int
    :param table_name: Tamano del dataset
    :type table_name: int
    :return: Dataset listo para ser usado por la red neuronal
    :rtype: tensorflow.python.data.ops.dataset_ops.TensorSliceDataset
  """

  AUTOTUNE = tf.data.AUTOTUNE

  dataset = dataset.shuffle(size)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=AUTOTUNE)
  dataset = dataset.cache()

  return dataset

**De que sirve la funcion anterior** [1 Pts] 

`Comentario`: 

In [ ]:
### START CODE HERE ###


**Explique el significado del tamano del batch (`batch size`) en la implementación moderna del algoritmo BP (`back-propagation`). ¿Qué valor recomendaría si su conjunto de entrenamiento es de 10.000 ejemplos? ¿Qué valor recomendaría si su conjunto de entrenamiento es de 1.000.000 de ejemplos?** [1 Pts]

`Respuesta:`

Finalmente, tenemos listo el dataset para ser utilizado por la red neuronal.

### Sequential API

Crear y entrenar una red neuronal que tenga:


*   1 capa de entrada (Identifique que dimensiones debería tener)
*   3 capas ocultas con 16 neuronas cada una y con funcion de activación `ReLu`
*   1 capa de salida con 1 neurona con funcion de activación por defecto (`linear`)


Usando [Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) [2 Pts]




*Para ello se puede apoyar en la función [Sequential](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) de `Tensorflow`.*

```
model = Sequential([
  Input(..., name=''),
  Dense(units=128, activation='relu', ..., name=''),
  Dense(units=128, activation='relu', ..., name=''),
  ...
  Dense(units=1, ..., name='')
])
```

Y compile el modelo utilizando model.compile de `Tensorflow` considerando como base el siguiente ejemplo:

```
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
```
Para mayor información se recomienda revisar el siguiente [link](https://www.tensorflow.org/guide/keras/train_and_evaluate#the_compile_method_specifying_a_loss_metrics_and_an_optimizer).

Utilizar:
* `optimizer` = `Adam(learning_rate=1e-3)` con los parámetros por defecto
* `loss` = `Mean Squared Error`
* `metrics` = `Mean Squared Error`

In [ ]:
### START CODE HERE ###


**Calcule la cantidad de parámetros entrenables de esta red y luego verifique su resultado utilizando `model.summary()`** (Cuando se pide el número de parámetros explicitar como llego a ese valor, que multiplicaciones o sumas tuvo que realizar) [0.5 Pts]

`Número de Parámetros` = ....

In [ ]:
### START CODE HERE ###


**Muestre la estructura de la red utilizando la función `tf.keras.utils.plot_model`** [1 Pts]

In [ ]:
### START CODE HERE ###


#### ¿Cómo crear funciones de perdida?

Para probar los siguientes apartados vamos a crear un dataset muy sencillo que corresponde a 100 datos extraídos de la siguiente ecuación:

$$y = 2x + 1$$

e intentaremos predecir $x=100$, lo cual debería ser $y=201$ o muy cercano a este valor.

In [ ]:
# Create a dummy dataset
x = tf.linspace(start=1, stop=100, num=100)
y = tf.multiply(x, 2) + 1

`Tensorflow` da bastante flexibilidad y podemos crear las funciones de pérdidas que queramos, en este caso creare la funcion de RMSE.


In [ ]:
# With subclassing
class MyCustomRMSEV1(Loss):

  def __init__(self, **kwargs):
    super(MyCustomRMSEV1, self).__init__(**kwargs)

  def call(self, y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    rmse = tf.math.sqrt(mse)
    return rmse

In [ ]:
model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='adam', loss=MyCustomRMSEV1())
model.fit(x, y, epochs=1000, verbose=0)

print(f"Prediction {model.predict([101.0])}")

Prediction [[203.01186]]


In [ ]:
# Without subclassing
def MyCustomRMSEV2():

  def loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    rmse = tf.math.sqrt(mse)
    return rmse

  return loss

In [ ]:
model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='adam', loss=MyCustomRMSEV2())
model.fit(x, y, epochs=1000, verbose=0)

print(f"Prediction {model.predict([101.0])}")

Prediction [[202.98393]]


En la versión que no utiliza subclassign le podemos pasar parámetros adicionales a la funcion, como por ejemplo:


In [ ]:
def MyCustomRMSEV3(factor):

  def loss(y_true, y_pred):
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    rmse = tf.math.sqrt(mse)
    return rmse / factor

  return loss

In [ ]:
model = Sequential([Dense(units=1, input_shape=[1])])
model.compile(optimizer='adam', loss=MyCustomRMSEV3(factor=100))
model.fit(x, y, epochs=1000, verbose=0)

print(f"Prediction {model.predict([101.0])}")

Prediction [[202.18678]]


**Existe una funcion de perdida para regresión llamada Huber, investigar cómo se calcula, impleméntela usando `Tensorflow` y prediga el valor para $x=100$ considerando el dataset dummy creado anteriormente** [1 Pts]




`Huber`: 

In [ ]:
### START CODE HERE ###


#### ¿Cómo crear funciones de activación?

También podemos crear funciones de activación utilizando `Tensorflow`. Para ello se muestra el ejemplo de la funcion de activación sigmoidal.

In [ ]:
def MySigmoid(x):
  return 1/ (1 + tf.math.exp(-x))

model = Sequential([Dense(units=1, input_shape=[1], activation=MySigmoid)])
model.compile(optimizer='adam', loss=MyCustomHuberLoss(delta=0.5))
model.fit(x, y, epochs=1000, verbose=0)

print(f"Prediction {model.predict([101.0])}")

Prediction [[258.6137]]


**Identifique la expresión matemática que computa la funcion de activación `LeakyRelu` e impleméntela utilizando `Tensorflow`. Utilice como funcion de activación `LeakyRelu` con cualquier valor de `alpha` y prediga el valor para $x=100$ considerando el dataset dummy creado anteriormente** [1 Pts]


In [ ]:
### START CODE HERE ###


`LeakyRelu`:

#### ¿Cómo crear capas personalizadas?

Para crear capas personalizadas tenemos que hacer subclassing a la clase `Layer`. En este caso se replica parte de la capa `Dense` que utilizamos anteriormente.

In [ ]:
class MyCustomDenseV1(Layer):

  def __init__(self, units=16, activation=None):
    """Inicializar los atributos de la instancia"""
    super(MyCustomDenseV1, self).__init__()
    self.units = units
    self.activation = tf.keras.activations.get(activation)

  def build(self, input_shape):
    """Crear el 'template' de la capa (pesos)"""
     # Initialize the weights
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(name="kernel",
                        initial_value=w_init(shape=(input_shape[-1], self.units), dtype='float32'),
                        trainable=True)
    b_init = tf.zeros_initializer()
    # Initialize the biases
    self.b = tf.Variable(name="bias",
                         initial_value=b_init(shape=(self.units, ), dtype='float32'),
                         trainable=True)
  
  def call(self, inputs):
    """Definir la operaciones que realiza la capa desde inputs a outputs"""
    return self.activation(tf.matmul(inputs, self.w) + self.b)

In [ ]:
model = Sequential([Input(shape=(1, )), MyCustomDenseV1(units=1)])
model.compile(optimizer=Adam(learning_rate=1e-3), loss=MyCustomRMSEV3(factor=10))
model.fit(x, y, epochs=500, verbose=0)

print(f"Prediction {model.predict([101.0])}")

Prediction [[201.69774]]


**Crear una capa densa con el nombre `MyCustomDenseV2` que compute lo siguiente: [1 Pts]
$$y = ax^3 * bx^2 - cx + d $$** 
Esto puede que no tenga mucho sentido en la práctica, es solo para que se familiaricen un poco con el código.

Utilice su capa `MyCustomDenseV2` y prediga el valor para $x=100$ considerando el dataset dummy creado anteriormente

In [ ]:
### START CODE HERE ###


**Calcular la cantidad de parámetros de su modelo anterior. Explicar su resultado** [0.5 Pts]

`Número de parámetros`: 

#### ¿Qué son los callbacks en TensorFlow?

**Existen multiples `Callbacks` dispibles en `Tensorflow` e incluso uno puede crear `Callbacks` personalizados. Defina la utilidad de los `Callbacks` y averiguar cómo funciona el callback [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) y [ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint). Finalmente cree una lista con ambos callbacks para posteriormente ser pasados al entrenamiento de la red** [1 Pts]

*   `Callbacks`:
*   `EarlyStopping`:
*   `ModelCheckpoint`:



Para `EarlyStopping` utilice los parámetros `monitor, patience y restore_best_weights`. Elija un valor apropiado para `monitor y patience`, el parámetro `restore_best_weights` déjelo en True.


Para `ModelCheckpoint` utilice los parámetros `filepath, monitor, save_best_only y save_weights_only`. Elija un valor para los parámetros `filepath y monitor`, los parámetros `save_best_only, save_weights_only` déjelos en True.


In [ ]:
### START CODE HERE ###


#### Entrenamiento de la primera red neuronal

Para entrenar una red la forma más sencilla es utilizar el metodo `model.fit`. También se puede personalizar lo que sucede en el model.fit pero eso quedara para el siguiente curso de introducción a las redes neuronales. Si les interesa saber acerca del tema les dejo el siguiente [link](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit)

Entrene su modelo utilizando `model.fit`. Defina un numero de epoch acorde al problema, utilice los callbacks definidos anteriormente y los dataset (entrenamiento y validacion) que ya tiene listos para ser pasados por la red. Guardar el resultado en una variable, puede ser llamada `history`.

In [ ]:
### START CODE HERE ###


**Grafique las curvas de entrenamiento. Comente el grafico** [1 Pts]

In [ ]:
### START CODE HERE ###


Identifique en que parte de la siguiente figura se encuentra su entrenamiento: 

<img src="https://drive.google.com/uc?id=1kecseaITvJeG45WNePflLn_u3sOYUyq4" width="50%"/>

**En caso de que se encuentre en la parte izquierda o derecha que tiene que recomendaría hacer?** [1 Pts]

`Respuesta`:

**Compute el resultado de su modelo para el conjunto de testing** [1 Pts]

In [ ]:
### START CODE HERE ### 


#### ¿Cómo seteamos los hiperparámetros en una red neuronal?

**Para setear los hiperparámetros en `Tensorflow` tenemos algo que se llama [Keras Tuner](https://www.tensorflow.org/tutorials/keras/keras_tuner). Investigue como utilizarlo y encuentre los mejores hiperparámetros para su modelo anterior** [2 Pts]

In [ ]:
### START CODE HERE ### 


**Muestre los mejores hiperparámetros obtenidos para la red neuronal, cree de nuevo la red y reentrénela usando los mejores hiperparámetros. Muestre las curvas de entrenamiento y finalmente evalúe su red con el conjunto de testing** [1 Pts]


In [ ]:
### START CODE HERE ### 


### Functional API

#### Entrenamiento

Crear y entrenar una red neuronal que tenga:


*   1 capa de entrada (Identifique que dimensiones debería tener)
*   3 capas ocultas con 16 neuronas cada una y con funcion de activación `ReLu`
*   1 capa de salida con 1 neurona con funcion de activación por defecto (`linear`)


usando [Functional API](https://www.tensorflow.org/guide/keras/functional?hl=es-419) [2 Pts]


Compilar el modelo de la misma forma que en el apartado de `Sequential API`

In [ ]:
### START CODE HERE ###


**Compruebe que al utilizar `Functional API` se obtiene la misma cantidad de parámetros entrenables que cuando se utiliza `Sequential API`. Realice el entrenamiento de modelo**  [1 Pts]


In [ ]:
### START CODE HERE ###


**Grafique la curva de entrenamiento y comente los resultados** [0.5 Pts]

In [ ]:
### START CODE HERE ###


Parece bastante similar `Sequential API` y `Functional API` la diferencia es que con `Functional API` podemos tener modelos más complejos que tengan por ejemplo skip connections o bien que tengan múltiples entradas y/o salidas (**Lo que puede o no servir para el desafío**)

**Investigar la utilidad de las skip connections en redes neuronales y agregue algunas skip connections al modelo anterior, puede agregar más capas si lo requiere** [2 Pts]

`Skip Connections`: 

In [ ]:
### START CODE HERE ###


**Muestre que su modelo si cuenta con skip connections utilizando `tf.keras.utils.plot_model`** [0.5 Pts]

In [ ]:
### START CODE HERE ###


**Entrene este modelo, grafique las curvas de entrenamiento y evalúe el modelo con el conjunto de testing** [0.5 Pts]


In [ ]:
### START CODE HERE ### 


#### Evaluación

In [ ]:
### START CODE HERE ###  


### Model Subclassing

#### Entrenamiento

Crear y entrenar una red neuronal que tenga:


*   1 capa de entrada (Identifique que dimensiones debería tener)
*   3 capas ocultas con 16 neuronas cada una y con funcion de activación `ReLu`
*   1 capa de salida con 1 neurona con funcion de activación por defecto (`linear`)


usando [Model Subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models) [2 Pts]

In [ ]:
### START CODE HERE ###  


Parece nuevamente que es lo mismo, pero la diferencia en este caso es que con esto podemos customizar el entrenamiento y muchas cosas más del modelo, pero hay que pagar el precio de trabajar a más bajo nivel.

**Entrene este modelo, grafique las curvas de entrenamiento y evalúe el modelo con el conjunto de testing. Comente los resultados** [0.5 Pts]

In [ ]:
### START CODE HERE ###


#### Evaluación

In [ ]:
### START CODE HERE ###


# 1.2) Ensamblados 📊



Típicamente son tres los métodos más comunes de ensamblado: `Bagging`, `Boosting` y `Stacking`.

**Defina cada uno de los métodos mencionados anteriormente** [2 Pts]


* `Bagging`:
* `Boosting`:
* `Stacking`: 

## Redes neuronales

**Cree un ensamblado de tipo `Bagging` utilizando los modelos creados con `Sequential API`, `Functional API` y `Model Subclassing`. Muestre el ensamble utilizando `tf.keras.utils.plot_model`, entrénelo, grafique las curvas de entrenamiento y evalúelo en el conjunto de testing. Comente los resultados** [4 Pts]

Recuerde congelar los parámetros de estos modelos luego de ser entrenados para ser utilizados en el ensamble, utilice `model.trainable=False` para congelar los parámetros.

In [ ]:
### START CODE HERE ###


## Modelos clásicos (Random Forest, XGBoost y CatBoost)

**Realizar un ensamble de tipo `Stacking` considerando el mismo dataset de house prediction. Para ello entrene 3 modelos distintos: `Random Forest`, `XGBoost` y `CatBoost`. Evalúe el ensamble en el conjunto de testing y comente los resultados** [4 Pts]

In [ ]:
### START CODE HERE ###


# 2) Desafío 🏆



[House Price Prediction](https://www.kaggle.com/t/82ab64ccf77248e9bce30924e6f58459)

<img src="https://drive.google.com/uc?id=1ep729kaGruTMz7Iz6MSnWrfyAy-y4OL9" width="60%"/>


**La nota de este apartado se hará en base a la posición del ranking obtenido en la plataforma `Kaggle`** [60 Pts]

*Todo el código tiene que estar presente en ese apartado incluyendo la generación del archivo para subirlo a la plataforma, pueden agregar la cantidad de celdas que requieran y se valora que incluyan comentarios a los resultados obtenidos* 

---
**Utilizar todo lo aprendido en las tareas incluyendo esta última**

In [ ]:
### START CODE HERE ###
